In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

The 1st 5 million rows of the training set is used for prediction. A quick look at the data set seems to tell that the data is in random order so no need to do random selection of 5 mil rows.

In [2]:
traindf = pd.read_csv('train.csv',nrows = 5000000)
testdf = pd.read_csv('test.csv')


In [3]:
traindf = traindf.drop(traindf[traindf.isnull().any(1)].index, axis = 0)

In [4]:
traindf = traindf.drop(traindf[traindf['fare_amount']<0].index, axis = 0)

In [5]:
traindf = traindf.drop(traindf[traindf['passenger_count']==208].index, axis = 0)

In [6]:
traindf = traindf.drop(traindf[traindf['pickup_latitude']<-90].index, axis = 0)
traindf = traindf.drop(traindf[traindf['pickup_latitude']>90].index, axis = 0)
traindf = traindf.drop(traindf[traindf['pickup_longitude']<-180].index, axis = 0)
traindf = traindf.drop(traindf[traindf['pickup_longitude']>180].index, axis = 0)

In [7]:
traindf = traindf.drop(traindf[traindf['dropoff_latitude']<-90].index, axis = 0)
traindf = traindf.drop(traindf[traindf['dropoff_latitude']>90].index, axis = 0)
traindf = traindf.drop(traindf[traindf['dropoff_longitude']<-180].index, axis = 0)
traindf = traindf.drop(traindf[traindf['dropoff_longitude']>180].index, axis = 0)

In [8]:
traindf['pickup_datetime']  = pd.to_datetime(traindf['pickup_datetime'])

In [9]:
testdf['pickup_datetime']  = pd.to_datetime(testdf['pickup_datetime'])

In [10]:
for a in [traindf,testdf]:
    a['Year'] = a['pickup_datetime'].dt.year
    a['Month'] = a['pickup_datetime'].dt.month
    a['Date'] = a['pickup_datetime'].dt.day
    a['Day of Week'] = a['pickup_datetime'].dt.dayofweek
    a['Hour'] = a['pickup_datetime'].dt.hour

In [11]:
testdf.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,5,11
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,5,21
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,5,21


In [12]:
testkey = testdf['key']

In [13]:
testdf = testdf.drop('key',axis = 1)
testdf = testdf.drop('pickup_datetime',axis = 1)
traindf = traindf.drop('key',axis = 1)
traindf = traindf.drop('pickup_datetime',axis = 1)

In [14]:
traindf["lonig_dist"] =  (traindf.dropoff_longitude - traindf.pickup_longitude).abs()
traindf["latit_dist"] =  (traindf.dropoff_latitude - traindf.pickup_latitude).abs()
testdf["lonig_dist"] =  (testdf.dropoff_longitude - testdf.pickup_longitude).abs()
testdf["latit_dist"] =  (testdf.dropoff_latitude - testdf.pickup_latitude).abs()

In [15]:
testdf.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,lonig_dist,latit_dist
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13,0.008110,0.019970
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13,0.012024,0.019817
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,5,11,0.002870,0.005121
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,5,21,0.009288,0.016172
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,5,21,0.022519,0.045348


In [16]:
traindf.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,lonig_dist,latit_dist
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,0,17,0.002701,0.009041
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,1,16,0.036780,0.070701
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,3,0,0.008504,0.010708
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,5,4,0.004437,0.024949
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,1,7,0.011440,0.015754


In [17]:
trainlables = traindf['fare_amount']

In [18]:
traindf = traindf.drop('fare_amount',axis =1 )

In [19]:
traindf.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,lonig_dist,latit_dist
0,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,0,17,0.002701,0.009041
1,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,1,16,0.036780,0.070701
2,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,3,0,0.008504,0.010708
3,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,5,4,0.004437,0.024949
4,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,1,7,0.011440,0.015754


In [20]:
trainFeatures = np.array(traindf)
trainLables = np.array(trainlables)
testFeatures = np.array(testdf)

In [21]:
scaler = StandardScaler()
scaler.fit(trainFeatures)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [22]:
trainFeatures = scaler.transform(trainFeatures)
print(trainFeatures)

[[-0.12873612  0.13152444 -0.12836599 ...  0.53547463 -0.05001452
  -0.04929215]
 [-0.14515962  0.12988629 -0.14154643 ...  0.3820144  -0.03944229
  -0.01336979]
 [-0.14197413  0.13805858 -0.14269292 ... -2.07334938 -0.04821427
  -0.04832098]
 ...
 [-0.14181395  0.13852553 -0.14154557 ...  1.45623605 -0.05029279
  -0.04422888]
 [-0.14031613  0.13771022 -0.14083959 ...  0.8423951  -0.04884062
  -0.04974118]
 [-0.14100414  0.1350994  -0.14025882 ... -0.84566749 -0.04875065
  -0.04801396]]


In [23]:
testFeatures = scaler.transform(testFeatures)

In [24]:
testkey = np.array(testkey)
print(testkey)

['2015-01-27 13:08:24.0000002' '2015-01-27 13:08:24.0000003'
 '2011-10-08 11:53:44.0000002' ... '2015-04-19 20:44:15.0000001'
 '2015-01-31 01:05:19.0000005' '2015-01-18 14:06:23.0000006']


In [25]:
trainFeatures =np.delete(trainFeatures, 8, 1)
testFeatures =np.delete(testFeatures, 8, 1)
trainFeatures =np.delete(trainFeatures, 7, 1)
testFeatures =np.delete(testFeatures, 7, 1)

In [26]:

#OLS 9.32658
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultOLS1.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()
    

In [113]:
#ridge 9.32823
from sklearn.linear_model import Ridge

reg = Ridge(tol = .00001).fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultRidge1.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()
    


In [29]:
#lasso 9.33031
from sklearn import linear_model

reg = linear_model.Lasso(alpha=0.1).fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultLasso.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [69]:
#ElasticNet 9.36910
from sklearn.linear_model import ElasticNet
reg = ElasticNet(random_state=0).fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultElasticNet.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [70]:
# BayesianRidge 9.32835
from sklearn import linear_model
reg = linear_model.BayesianRidge().fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultBayes.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

# Code for  best submission

In [117]:
# RandomForestRegressor 3.37778
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators = 10, n_jobs = -1).fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultRFR.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [121]:
# AdaBoostRegressor linear 5.16535
from sklearn.ensemble import AdaBoostRegressor
reg = AdaBoostRegressor(n_estimators = 50,loss= 'linear').fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultAdaBLin.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [122]:
# AdaBoostRegressor square 4.87054
from sklearn.ensemble import AdaBoostRegressor
reg = AdaBoostRegressor(n_estimators = 50,loss= 'square').fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultAdaBSquare.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [123]:
# AdaBoostRegressor exp 4.97820
from sklearn.ensemble import AdaBoostRegressor
reg = AdaBoostRegressor(n_estimators = 50,loss= 'exponential').fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultAdaBExponential.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

# Code for 2nd best submission

In [124]:
# GradientBoostingRegressor 3.50074
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor().fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultGradientBoostingRegressor.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

In [126]:
# xgboost 3.68642
import xgboost as xgb
reg = xgb.XGBRegressor(n_jobs = -1).fit(trainFeatures, trainLables)
ans = reg.predict(testFeatures)

f = open("resultXGB.csv", "w")
f.write("key,fare_amount\n")
for i in range(len(testkey)):
    f.write("%s,%s\n" % (testkey[i],ans[i]))
f.close()

[21:49:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:49:40] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
